# Introduction to data science
Author: Gérard Lichtert

## Introduction
This notebook is to clean data from a csv, it removes unnecesary columns, computes means and saves the processed data to a new csv file found in the output folder.

It will also make a new dataframe containing averages per day per participant and save it to a csv

## Variables you can change
In the following code cell you can change the variables as you need

In [1]:
# * This is a list of headers we want to delete (exluding the ones with _TZ, _RT and _TZ)
# * the headers with _TZ, _RT and _TZ will be removed automatically.
HEADERS_TO_DELETE: list[str] = [
    "STUDY_ID",
    "STUDY_NAME",
    "STUDY_VERSION",
    "SURVEY_ID",
    "TRIGGER",
    "START_END",
    "RAND_PROB",
    "CONTROLLEVRAAG",
    "INTRO",
    "SLOT",
    "BEDANKT",
    "INLEIDING",
]

In [2]:
from obse import obse

csv = obse.load_csv("exports_8gQqgGGHa_export_F005kvQNV.csv")
csv = obse.process_obse(csv, HEADERS_TO_DELETE, False)
print(csv.collect())
max = obse.create_daily_means(csv)
print(max.collect())

shape: (5, 15)
┌────────────────┬─────────────┬───────────────────┬────────────┬───┬─────┬─────┬─────┬──────┐
│ PARTICIPANT_ID ┆ SURVEY_NAME ┆ COMPLETED_TS      ┆ ACTIVITEIT ┆ … ┆ 8   ┆ 9   ┆ 10  ┆ mean │
│ ---            ┆ ---         ┆ ---               ┆ ---        ┆   ┆ --- ┆ --- ┆ --- ┆ ---  │
│ str            ┆ str         ┆ str               ┆ i64        ┆   ┆ u64 ┆ u64 ┆ u64 ┆ f64  │
╞════════════════╪═════════════╪═══════════════════╪════════════╪═══╪═════╪═════╪═════╪══════╡
│ s782393640     ┆ OBSE        ┆ 23-Apr-2024 18:08 ┆ 1          ┆ … ┆ 2   ┆ 1   ┆ 1   ┆ 1.7  │
│ s782393640     ┆ OBSE        ┆ 22-Apr-2024 21:01 ┆ 1          ┆ … ┆ 3   ┆ 2   ┆ 5   ┆ 3.3  │
│ s782393640     ┆ OBSE        ┆ 24-Apr-2024 21:04 ┆ 1          ┆ … ┆ 3   ┆ 2   ┆ 4   ┆ 2.8  │
│ s782393640     ┆ OBSE        ┆ 20-Apr-2024 18:01 ┆ 1          ┆ … ┆ 2   ┆ 2   ┆ 1   ┆ 1.6  │
│ s782393640     ┆ OBSE        ┆ 19-Apr-2024 18:01 ┆ 1          ┆ … ┆ 3   ┆ 2   ┆ 2   ┆ 2.2  │
└────────────────┴─────────────┴───